In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [3]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [4]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [5]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [6]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [9]:
txt = "awesome product. I love it"
preprocessed_txt = preprocessing(txt)

In [10]:
preprocessed_txt

0    awesom product love
Name: tweet, dtype: object

In [11]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [12]:
vectorized_txt= vectorizer(preprocessed_txt,tokens)

In [13]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
model.predict(vectorized_txt)

array([0], dtype=int64)

In [15]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [16]:
txt1 = "very nice project. I like and love it"
preprocessed_txt1 = preprocessing(txt1)
vectorized_txt1 = vectorizer(preprocessed_txt1, tokens)
prediction = get_prediction(vectorized_txt1)
prediction

'positive'

In [17]:
model.predict(vectorized_txt1)

array([0], dtype=int64)

In [18]:
txt2 = "very bad project. I hate it"
preprocessed_txt2 = preprocessing(txt2)
vectorized_txt2 = vectorizer(preprocessed_txt2, tokens)
prediction = get_prediction(vectorized_txt2)
prediction

'negative'

In [19]:
model.predict(vectorized_txt2)

array([1], dtype=int64)